<a href="https://colab.research.google.com/github/Telurio27/Python---Programacion-orientada-a-objetos/blob/main/EXAMEN_3_PYTHON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***EVALUACION 2***
PYTHON APLICADO A INGENIERIA

*   ANGEL DAVID TENORIO LOBO

DOCENTE: MIGUEL ORTIZ



In [53]:

import numpy as np

# Cargar el archivo en la variable data con sus respectivos parametros
data = np.genfromtxt("GHI_Temp_Daily_Hourly_2024.csv", delimiter=";",skip_header=1)

print("Tamaño del archivo (filas, columnas):", data.shape)

# Imprimir filas y columnas por separado
print("Número de filas:", data.shape[0])
print("Número de columnas:", data.shape[1])

#creamos variables para filtrar
month = 1
day = 2
year = 2024

mask = (data[:,0] == year) & (data[:,1] == month) & (data[:,2] == day) #filtracion de los datos
filtered_data = data[mask] #añadimos a la variable los datos filtrados

print(filtered_data) #imprimir datos filtrados


# Función para filtrar y exportar datos
import numpy as np

# Función para filtrar y exportar datos
def filter_data_by_date(filepath: str, year: int, month: int, day: int):
    try:
        # Cargar los datos del archivo
        data = np.genfromtxt(filepath, delimiter=";", skip_header=1)
        print(f" Se cargaron {data.shape[0]} registros del archivo '{filepath}'.")

        # Estadísticas generales
        print(f" Temperatura promedio: {np.mean(data[:,4]):.2f} °C")
        print(f" GHI máximo: {np.max(data[:,5]):.2f} W/m2")

        # Crear máscara booleana
        mask = (data[:,0] == float(year)) & (data[:,1] == float(month)) & (data[:,2] == float(day))
        filtered_data = data[mask]

        # Validación: si no hay resultados
        if filtered_data.shape[0] == 0:
            print(f"No se encontraron datos para la fecha 😒 {year}-{month:02d}-{day:02d}.")
            return None

        # Crear nombre del archivo con formato solicitado
        filename = f"filtered_data_{year}_{month:02d}_{day:02d}.csv"

        # Escribir el archivo CSV
        with open(filename, "w") as f:
            # Encabezado
            f.write("Year;Month;Day;Hour;Temperature;GHI\n")

            # Filas de datos
            for row in filtered_data:
                line = f"{int(row[0])};{int(row[1])};{int(row[2])};{int(row[3])};{row[4]:.2f};{row[5]:.2f}\n"
                f.write(line)

        # Confirmación
        print(f" Archivo '{filename}' creado exitosamente con {filtered_data.shape[0]} filas.")
        return filename

    # Manejo de errores
    except FileNotFoundError:
        print(f" Error: No se encontró el archivo '{filepath}'.")
    except IndexError:
        print(" Error: El archivo no tiene el formato esperado (faltan columnas).")
    except Exception as e:
        print(f" Error inesperado: {e}")



# Pruebas finales
import numpy as np

import numpy as np

if __name__ == "__main__":
    # PRUEBA: Fecha válida (1 de enero de 2024)
    filename = filter_data_by_date("GHI_Temp_Daily_Hourly_2024.csv", 2024, 1, 1)

    if filename:
        # Cargar los datos filtrados en ghi_temp_data
        ghi_temp_data = np.genfromtxt(filename, delimiter=";", skip_header=1)

        # Extraer columnas necesarias
        hours = ghi_temp_data[:, 3]   # Columna Hour
        temp = ghi_temp_data[:, 4]    # Columna Temperature
        ghi  = ghi_temp_data[:, 5]    # Columna GHI

        # Paso 3: Parámetros del sistema PV
        stc_temp = 25                       # Temperatura estándar (°C)
        temp_coeff = -0.004                 # Coeficiente de temperatura
        panel_area = 1.65                    # Área de un panel (m²)
        panel_efficiency = 0.20              # Eficiencia del panel (20%)
        n_panels = 10                        # Número de paneles
        total_area = panel_area * n_panels   # Área total (m²)
        inverter_efficiency = 0.95           # Eficiencia del inversor

        # Paso 4: Calcular corrección por temperatura
        temp_correction = 1 + temp_coeff * (temp - stc_temp)
        print("\nCorrección por temperatura calculada.")
        print("Ejemplo primeros valores:", temp_correction[:5])

        # Paso 5: Calcular potencia DC y AC
        dc_power = ghi * total_area * panel_efficiency * temp_correction
        ac_power = dc_power * inverter_efficiency

        print("\nPotencia DC (primeros valores):", dc_power[:5])
        print(" Potencia AC (primeros valores):", ac_power[:5])

        # Paso 6: Calcular energía generada
        hourly_energy_kwh = ac_power / 1000   # pasar a kWh
        total_energy_kwh = np.sum(hourly_energy_kwh)

        print("\nEnergía horaria (primeros valores en kWh):", hourly_energy_kwh[:5])
        print(f"Energía total generada: {total_energy_kwh:.2f} kWh")

        # Crear archivo con resultados
        output_filename = f"energy_data_2024_01_01.csv"
        with open(output_filename, "w") as datafile:
            datafile.write("Hour,Power_DC,Power_AC,Energy_kWh\n")
            for i in range(len(hours)):
                datafile.write(
                    f"{int(hours[i])},{dc_power[i]:.2f},{ac_power[i]:.2f},{hourly_energy_kwh[i]:.2f}\n"
                )

        # Mensajes informativos
        print("\nArchivo creado:", output_filename)
        print("Número de registros:", len(hours))
        print(f"Potencia DC promedio: {np.mean(dc_power):.2f} W")
        print(f"Potencia AC promedio: {np.mean(ac_power):.2f} W")
        print(f"Energía total producida: {total_energy_kwh:.2f} kWh")

def energy_calculations(filepath: str, year: int, month: int, day: int,
                        n_panels: int, panel_area: float, panel_efficiency: float,
                        inverter_efficiency: float,
                        stc_temp: float = 25, temp_coeff: float = -0.004):
    try:
        # Archivos de salida
        filtered_data_filename = f"filtered_data_{year}_{month:02d}_{day:02d}.csv"
        energy_data_filename = f"energy_data_{year}_{month:02d}_{day:02d}.csv"

        # Filtrar los datos para la fecha solicitada
        result_file = filter_data_by_date(filepath, year, month, day)
        if not result_file:
            print(f"No se pudo generar el archivo filtrado para {year}-{month:02d}-{day:02d}.")
            return None

        # Cargar datos filtrados
        data = np.genfromtxt(filtered_data_filename, delimiter=";", skip_header=1)
        hours = data[:, 3]
        temp = data[:, 4]
        ghi = data[:, 5]

        # Área total del sistema
        total_area = n_panels * panel_area

        # Corrección por temperatura
        temp_correction = 1 + temp_coeff * (temp - stc_temp)

        # Potencia DC y AC
        dc_power = ghi * total_area * panel_efficiency * temp_correction
        ac_power = dc_power * inverter_efficiency

        # Energía horaria en kWh
        hourly_energy_kwh = ac_power / 1000
        total_energy_kwh = np.sum(hourly_energy_kwh)

        # Crear archivo de resultados
        with open(energy_data_filename, "w") as f:
            f.write("Hour,Power_DC,Power_AC,Energy_kWh\n")
            for i in range(len(hours)):
                f.write(f"{int(hours[i])},{dc_power[i]:.2f},{ac_power[i]:.2f},{hourly_energy_kwh[i]:.2f}\n")

        # Mensajes informativos
        print(f"Archivo creado: {energy_data_filename}")
        print(f"Número de registros: {len(hours)}")
        print(f"Potencia DC promedio: {np.mean(dc_power):.2f} W")
        print(f"Potencia AC promedio: {np.mean(ac_power):.2f} W")
        print(f"Energía total producida: {total_energy_kwh:.2f} kWh")

        return energy_data_filename

    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filepath}.")
    except IndexError:
        print("Error: El archivo no tiene el formato esperado.")
    except Exception as e:
        print(f"Error inesperado: {e}")


# Paso 8: Pruebas
if __name__ == "__main__":
    # Configuración del sistema
    n_panels = 10
    panel_area = 1.65
    panel_efficiency = 0.20
    inverter_efficiency = 0.95

    print("\nPRUEBA 1: Fecha válida (1 de enero de 2024)")
    energy_calculations("GHI_Temp_Daily_Hourly_2024.csv", 2024, 1, 1,
                        n_panels, panel_area, panel_efficiency, inverter_efficiency)

    print("\nPRUEBA 2: Fecha inexistente (30 de febrero de 2024)")
    energy_calculations("GHI_Temp_Daily_Hourly_2024.csv", 2024, 2, 30,
                        n_panels, panel_area, panel_efficiency, inverter_efficiency)

    print("\nPRUEBA 3: Archivo inexistente")
    energy_calculations("archivo_inexistente.csv", 2024, 1, 1,
                        n_panels, panel_area, panel_efficiency, inverter_efficiency)

    print("\nPRUEBA 4: Otra fecha válida (2 de enero de 2024)")
    energy_calculations("GHI_Temp_Daily_Hourly_2024.csv", 2024, 1, 2,
                        n_panels, panel_area, panel_efficiency, inverter_efficiency)

    print("\nPRUEBA 5: Configuración diferente (más paneles, 3 de enero de 2024)")
    energy_calculations("GHI_Temp_Daily_Hourly_2024.csv", 2024, 1, 3,
                        20, panel_area, panel_efficiency, inverter_efficiency)



energy_calculations("GHI_Temp_Daily_Hourly_2024.csv", 2024, 12, 11, 10,
1.65, 0.2, 0.95)


Tamaño del archivo (filas, columnas): (8785, 6)
Número de filas: 8785
Número de columnas: 6
[[2.024e+03 1.000e+00 2.000e+00 0.000e+00 2.720e+01 0.000e+00]
 [2.024e+03 1.000e+00 2.000e+00 1.000e+00 2.670e+01 0.000e+00]
 [2.024e+03 1.000e+00 2.000e+00 2.000e+00 2.640e+01 0.000e+00]
 [2.024e+03 1.000e+00 2.000e+00 3.000e+00 2.610e+01 0.000e+00]
 [2.024e+03 1.000e+00 2.000e+00 4.000e+00 2.580e+01 0.000e+00]
 [2.024e+03 1.000e+00 2.000e+00 5.000e+00 2.560e+01 0.000e+00]
 [2.024e+03 1.000e+00 2.000e+00 6.000e+00 2.550e+01 0.000e+00]
 [2.024e+03 1.000e+00 2.000e+00 7.000e+00 2.550e+01 0.000e+00]
 [2.024e+03 1.000e+00 2.000e+00 8.000e+00 2.550e+01 0.000e+00]
 [2.024e+03 1.000e+00 2.000e+00 9.000e+00 2.560e+01 0.000e+00]
 [2.024e+03 1.000e+00 2.000e+00 1.000e+01 2.560e+01 0.000e+00]
 [2.024e+03 1.000e+00 2.000e+00 1.100e+01 2.580e+01 0.000e+00]
 [2.024e+03 1.000e+00 2.000e+00 1.200e+01 2.750e+01 9.200e+01]
 [2.024e+03 1.000e+00 2.000e+00 1.300e+01 2.960e+01 9.300e+01]
 [2.024e+03 1.000e+00 2.00

'energy_data_2024_12_11.csv'